In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

True

In [2]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings 

# ----- OpenAI ----- #
chat_model = ChatOpenAI()
embeddings = OpenAIEmbeddings()

# ----- Ollama ----- #
# chat_model = ChatOllama()
# embeddings = OllamaEmbeddings()


In [3]:
import os
from pinecone import Pinecone as PineconeClient
from langchain_community.vectorstores import Pinecone

pinecone_client = PineconeClient(
   api_key=os.getenv("PINECONE_API_KEY"),
   environment=os.getenv("PINECONE_ENVIRONMENT")
)

vecdb = Pinecone.from_existing_index(
    index_name="sot-demo",
    embedding=embeddings,
)

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

from langchain.tools.retriever import create_retriever_tool


import gradio as gr

vecdb_tool = create_retriever_tool(
    vecdb.as_retriever(),
    "search_vecdb",
    "Searches the vector database for similar questions and returns the most similar texts.",
)

tools = load_tools(["wikipedia", "arxiv"], llm=chat_model)
tools.append(vecdb_tool)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools,
    chat_model,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors="Check your output and make sure it conforms!",
    memory=memory
)

def call_agent(user_question):
    response = agent.run(input=user_question)
    return response

with gr.Blocks() as demo:
    title = gr.HTML("<h1>The Data Moves Me Chatbot</h1>")
    input = gr.Textbox(label="What would you like to know?")
    output = gr.Textbox(label="Answer")
    btn = gr.Button("Ask")
    btn.click(fn=call_agent, inputs=input, outputs=output)

demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://375df7ff7aeb68ca2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
```json
{
    "action": "wikipedia",
    "action_input": "AI Agents"
}
```
Observation: Page: Intelligent agent
Summary: In intelligence and artificial intelligence, an intelligent agent (IA) is an agent acting in an intelligent manner; It perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge. An intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.
Leading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term borrowed from economics, "rational agent".An agent has an "objective function" that encapsu